# Módulo 10 atividade 1

Uma instituição financeira quer conhecer melhor o perfil de renda de seus novos clientes para diversos fins, por exemplo, melhor dimensionar o limite de cartões de crédito dos novos clientes, sem necessariamente solicitar olerites ou documentações que impactem na experiência do seu cliente.

Para isto, conduziu um estudo com alguns clientes, comprovando suas rendas através de olerites e outros documentos, e pretende construir um modelo preditivo para esta renda com base em algumas variáveis que já possui em seu banco de dados.

As variáveis são intuitivas - note que há uma variável 'index' que é um identificador do cliente e que em geral o ```read_csv``` do pandas coloca também uma variável sequencial.

Estes dados estão no arquivo ```previsao_de_renda.csv```. Carregue-o em um *dataframe*.

1) Avalie a estrutura de correlação das variáveis quantitativas através de um gráfico de "matriz de dispersão" e através da avaliação gráfica da matriz de correlações. Veja se você identifica algum padrão interessante ou que te faça sentido.

In [161]:
import pandas as pd
        
previsao_de_renda = pd.read_csv('../../../../Datasets/previsao_de_renda.csv', sep = ',')
previsao_de_renda

In [162]:
# dropando a coluna index   
previsao_de_renda.drop(columns=['index','Unnamed: 0'], inplace=True)

In [163]:
# array com variáveis quantitativas
variaveis_quantitativas = previsao_de_renda.select_dtypes(include=['float64', 'int64']).columns

In [164]:
# Matriz de dispersão
import seaborn as sns
sns.pairplot(previsao_de_renda[variaveis_quantitativas])

In [165]:
previsao_de_renda[variaveis_quantitativas].corr()   

    2) Avalie um gráfico de dispersão (*scatterplot*) específico para as duas variáveis mais correlacionadas com *renda*.

In [166]:
import seaborn as sns

# Scatterplot for 'tempo_emprego' vs 'renda'
sns.scatterplot(x='tempo_emprego', y='renda', data=previsao_de_renda)

In [183]:
_ = sns.regplot(x='tempo_emprego', y='renda', data=previsao_de_renda)

In [167]:
# Scatterplot for 'idade' vs 'renda'
sns.scatterplot(x='idade', y='renda', data=previsao_de_renda)

In [184]:
_ = sns.regplot(x='idade', y='renda', data=previsao_de_renda)

3) Na su aopinião, há outliers na variável renda?

Método 1: Identificação de outliers através de z-score

In [168]:
import numpy as np
data= previsao_de_renda['renda']
from scipy import stats 

# Calcula a pontuação Z
z_scores = np.abs(stats.zscore(data))

# Identifica onde a pontuação Z é maior que 3 ou menor que -3
outliers = data[z_scores > 3]

print("Outliers:", outliers.count())

Método 2: Identificação de outliers através do IQR

In [169]:

Q1 = np.percentile(data, 25)
Q3 = np.percentile(data, 75)
IQR = Q3 - Q1

# Identifica outliers
outliers = data[(data < Q1 - 1.5 * IQR) | (data > Q3 + 1.5 * IQR)]

print("Outliers:", outliers.count())


Método 3: Identificação de outliers através de Isolation Forest

In [170]:
from sklearn.ensemble import IsolationForest

# Ajuste do modelo usando o DataFrame diretamente, mantendo o nome da coluna
clf = IsolationForest(random_state=0).fit(previsao_de_renda[['renda']])

# Uso da previsão mantendo a estrutura do DataFrame
outliers = previsao_de_renda[clf.predict(previsao_de_renda[['renda']]) == -1]

print("Outliers:", outliers['renda'].count())

Método 4: Identificação de outliers através de DBSCAN   

In [171]:
from sklearn.cluster import DBSCAN

clustering = DBSCAN(eps=3, min_samples=2).fit(previsao_de_renda[['renda']].values.reshape(-1, 1))
outliers = previsao_de_renda['renda'][clustering.labels_ == -1]

print("Outliers:", outliers.count())

Resposta: Sim, há muitos outliers na variável renda.

4) Calcule o logaritmo da variável renda e repita a sequência de análise

In [172]:
# logaritmo da variável renda
import numpy as np
previsao_de_renda['log_renda'] = np.log(previsao_de_renda['renda'])
# adicionar a variável log_renda ao array de variáveis quantitativas
variaveis_quantitativas = previsao_de_renda.select_dtypes(include=['float64', 'int64']).columns

In [173]:
# repetindo a sequência de análise
sns.pairplot(previsao_de_renda[variaveis_quantitativas])

In [174]:
previsao_de_renda[variaveis_quantitativas].corr()

In [175]:
# Scatterplot for 'tempo_emprego' vs 'renda'
sns.scatterplot(x='tempo_emprego', y='log_renda', data=previsao_de_renda)

In [176]:
_ = sns.regplot(x='tempo_emprego', y='log_renda', data=previsao_de_renda)

In [177]:
# Scatterplot for 'idade' vs 'renda'
sns.scatterplot(x='idade', y='log_renda', data=previsao_de_renda)

In [178]:
_ = sns.regplot(x='idade', y='log_renda', data=previsao_de_renda)

In [179]:
_ = sns.regplot(x='qt_pessoas_residencia', y='log_renda', data=previsao_de_renda)

5) A aplicação do LOG você acha que melhorou ou piorou a análise? 
 qt_pessoas_residencia foi a que mais aumentou a correlação com a renda após a aplicação do log. fora isso, não houve muita diferença.

In [180]:
# utilizando rank
df_rank = pd.concat([previsao_de_renda[variaveis_quantitativas], previsao_de_renda[variaveis_quantitativas].rank()], axis=1)

In [181]:
print(df_rank.columns)  
# colocando '_rank' no nome das colunas da metade para frente 
df_rank.columns = list(df_rank.columns[:len(variaveis_quantitativas)]) + [str(col) + '_rank' for col in df_rank.columns[len(variaveis_quantitativas):]]
print(df_rank.columns)



In [182]:
df_rank.corr()

 Com o rank não houve melhora na análise
 parece realmente que a única variável que tem correlação com a renda é o tempo de emprego.